In [2]:
import torch

torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [3]:
!nvidia-smi

Sat Jun  7 11:02:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.61                 Driver Version: 572.61         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   41C    P8              3W /   60W |     785MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import json
from transformers import pipeline
from datetime import datetime
import re

# Load the processed JSON data
with open('processed_reviews.json', 'r') as file:
    reviews = json.load(file)

In [5]:
pipe = pipeline("text-classification", model="nlptown/bert-base-multilingual-uncased-sentiment")

Device set to use cuda:0


In [7]:
for review in reviews:
    # Perform sentiment analysis on the cleaned text with truncation
    text = review['text_cleaned']
    try:
        result = pipe(text, truncation=True, max_length=512)[0]
        # The model outputs labels like '1 star', '2 stars', etc. Extract the star rating
        star_label = result['label']
        review['sentiment_stars'] = int(star_label.split()[0])  # Extract number (1-5)
        review['sentiment_score'] = result['score']  # Confidence score (0 to 1)
    except Exception as e:
        # Handle any errors (e.g., empty or invalid text)
        print(f"Error processing review '{review['title_cleaned']}': {e}")
        review['sentiment_stars'] = None
        review['sentiment_score'] = None

# Save the processed data with sentiment results
with open('sentiment.json', 'w') as file:
    json.dump(reviews, file, indent=2)

# Optional: Print results to verify
for review in reviews:
    print(f"Title: {review['title_cleaned']}")
    print(f"Text: {review['text_cleaned']}")
    print(f"Date: {review['date_cleaned']}")
    print(f"Verified Purchase: {review['verified_purchase_bool']}")
    print(f"Helpful Votes: {review['helpful_votes_count']}")
    print(f"Sentiment: {review['sentiment_stars']} stars (Confidence: {review['sentiment_score']:.3f})")
    print("---")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Title: amazons's basics 3-button usb wired mouse, works good, looks good, reduces the emf
Text: this amazon basics 3-button usb wired mouse is easy to review. it was delivered in a solid, well protected box. installing it was easy. it fit well, functions well, is responsive, and the design reminds me of when computer mouses were first introduced, so the familiarity is refreshing. it tracks well, without too fast or slow a speed and doesn't need batteries. i bought it because it is wired and helps reduce the emf around the computer from 20 mg with a wireless mouse to 0 mg for this one.
Date: 2025-05-16
Verified Purchase: True
Helpful Votes: 2
Sentiment: 5 stars (Confidence: 0.690)
---
Title: quick and easy
Text: this is simply wonderful. all of a sudden one day, i was unable to scroll with my mouse. what a pain! i'd had that one for 2 years maybe. i'd say i got my money's worth. i ordered the same one and it was here in one or two days, couldn't get any simpler.
Date: 2025-05-28
Verifie